# Fitting ellipsoids to labelled volume data

This notebook demonstrates how to fit ellipsoids to labelled volume data and then
compare the ellipsoids using various calculations and statistics.

First get a labelled volume from the example data (or somewhere else):

In [1]:
from mouse_embryo_labeller import tools
folder = "../" + tools.EXAMPLE_FOLDER
folder

'../../example_data/'

In [2]:
nc = tools.get_example_nucleus_collection(folder)
tsc = tools.get_example_timestamp_collection(folder, nc)
tsc.id_to_timestamp.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [3]:
ts = tsc.id_to_timestamp[10]
ts.load_truncated_arrays()
label_array = ts.l3d_truncated
label_array.shape

(71, 180, 247)

In [4]:
# The label array contains a small number of labels in a numpy int array:
import numpy as np
np.unique(label_array)

array([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [5]:
# View the raw array.  Slide the slider to move between layers.
from jp_doodle import array_image
array_image.show_arrays(label_array)

In [6]:
# Create an array fitter object for this array and adjust the coordinates
# so the array maps to a geometric unit cube, using di, dj, dk.

from mouse_embryo_labeller.ellipsoid_fit import ArrayFitter

(I, J, K) = label_array.shape
AF = ArrayFitter(label_array, di=1.0/I, dj=1.0/J, dk=1.0/K)

In [7]:
# Fit all labels (>0) inside a single ellipse and display a jupyter widgets graphic of the fit.
# Use 1000 randomly chosen test points for the fit.

info = AF.fit_ellipse_to_range(lower_limit=1, upper_limit=None, display=True, point_limit=1000)

In [8]:
# Drag the mouse in the generated graphic to rotate the figures.

In [9]:
print("Statistics")
print("axes", info.axes())
print("axis lengths", info.axis_lengths())
print("volume", info.volume(), "surface_area", info.surface_area(), "sphericity", info.sphericity())
print("center", info.center)

Statistics
axes [[ 0.17407219 -0.07438457 -0.00227515]
 [ 0.12016935  0.28039302  0.0269119 ]
 [-0.00447138 -0.01625435  0.18931886]]
axis lengths [0.189312882310212, 0.30624363851760283, 0.19006796075560514]
volume 0.046157764628599114 surface_area 0.6457129835125743 sphericity 0.963698355056072
center [0.36898536 0.614798   0.49575449]


In [10]:
# Fit ellipses to each labelled region separately (don't display widgets).

label_to_info = {}
for i in np.unique(label_array):
    if i > 0:
        label_to_info[i] = AF.fit_ellipse_to_range(lower_limit=i, upper_limit=i, point_limit=1000)

In [11]:
# Show relationship of each label ellipse to the ellipse containing all labels.
for i in sorted(label_to_info):
    label_info = label_to_info[i]
    print(
        i,
        "offset", 
        label_info.relative_offset_to_center_of(info),
        "inside?", 
        label_info.proportion_inside_of(info),
        "to center",
        label_info.center_distance_to_center_of(info),
        "to surface",
        label_info.center_distance_to_surface_of(info)
    )

1 offset 0.772651889042694 inside? 0.9241803278688525 to center 0.14669466606318132 to surface 0.043164011749584015
2 offset 0.8626721741700178 inside? 0.9651639344262296 to center 0.23787351208412869 to surface 0.03786682034630845
3 offset 0.7678366477950722 inside? 1 to center 0.14765193902021007 to surface 0.044644090928619526
4 offset 0.7788932340507121 inside? 0.9836065573770493 to center 0.19984887321434966 to surface 0.05673170096141454
5 offset 0.7483263034515352 inside? 1 to center 0.1995677411328054 to surface 0.06711771441289853
6 offset 0.8158422230196257 inside? 0.9672131147540984 to center 0.15816658591505672 to surface 0.035702499861902236
7 offset 0.6638749072398963 inside? 1 to center 0.13174035541021356 to surface 0.06670117924265168
8 offset 0.8777426219733535 inside? 0.889344262295082 to center 0.2512297581337347 to surface 0.034992822204128284


In [12]:
print(info.relative_offset_to_center_of.__doc__)


        Linear measure of how near the center of this ellipse is to the center of the other
        ellipse.  Returns 0 at the center, 1 at the boundary of the ellipse and > 1 outside the ellipse.
        


In [13]:
print(info.proportion_inside_of.__doc__)


        Return approximate proportion of this ellipse inside the other ellipse by relative volume.
        Returns 1 if completely inside and 0 if there is (nearly) no intersection.
        


In [14]:
print(info.center_distance_to_center_of.__doc__)

Distance between the center of this ellipsoid to the center of the other ellipsoid.


In [15]:
print(info.center_distance_to_surface_of.__doc__)

Distance between the center of this ellipsoid to the projection of the center onto the surface of the other ellipsoid.
